# Using Pandas for Time Series Analysis


In [ ]:
!pip install yfinance
import yfinance as yf


/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


# Get data for SPY -- S&P 500 ETF.

In [ ]:
SPY=yf.download(['SPY'])

[*********************100%%**********************]  1 of 1 completed


In [ ]:
type(SPY)

pandas.core.frame.DataFrame

In [ ]:
SPY

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-01-29,43.968750,43.968750,43.750000,43.937500,24.840683,1003200
1993-02-01,43.968750,44.250000,43.968750,44.250000,25.017357,480500
1993-02-02,44.218750,44.375000,44.125000,44.343750,25.070356,201300
1993-02-03,44.406250,44.843750,44.375000,44.812500,25.335382,529400
1993-02-04,44.968750,45.093750,44.468750,45.000000,25.441381,531500
...,...,...,...,...,...,...
2024-02-09,498.839996,501.649994,498.489990,501.200012,501.200012,63979400
2024-02-12,501.170013,503.500000,500.239990,500.980011,500.980011,56502300
2024-02-13,494.529999,497.089996,490.720001,494.079987,494.079987,113099200


Why does "Close" and "Adj Close" differ at "the beginning of time"? Notice that in recent years, "Close" and "Adj Close" are the same. So why did it differ before?

Let's graph it out to find out.



In [ ]:
import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"
SPY[['Adj Close','Close']].plot()

# Adj Close and Close
* Why is Adj Close and Close different far back in time, but same today?


Maybe it will help if we calculate returns.

2 ways:
today's price divided by yesterday's price.

In [ ]:
SPY['Adj Close']/SPY['Adj Close'].shift(1)

Date
1993-01-29         NaN
1993-02-01    1.007112
1993-02-02    1.002119
1993-02-03    1.010571
1993-02-04    1.004184
                ...   
2024-02-09    1.005779
2024-02-12    0.999561
2024-02-13    0.986227
2024-02-14    1.009088
2024-02-15    1.006799
Name: Adj Close, Length: 7818, dtype: float64

In [ ]:
SPY['Adj Close']/SPY['Adj Close'].shift(1)-1

Date
1993-01-29         NaN
1993-02-01    0.007112
1993-02-02    0.002119
1993-02-03    0.010571
1993-02-04    0.004184
                ...   
2024-02-09    0.005779
2024-02-12   -0.000439
2024-02-13   -0.013773
2024-02-14    0.009088
2024-02-15    0.006799
Name: Adj Close, Length: 7818, dtype: float64

In [ ]:
SPY['Adj Close'].pct_change()

Date
1993-01-29         NaN
1993-02-01    0.007112
1993-02-02    0.002119
1993-02-03    0.010571
1993-02-04    0.004184
                ...   
2024-02-09    0.005779
2024-02-12   -0.000439
2024-02-13   -0.013773
2024-02-14    0.009088
2024-02-15    0.006799
Name: Adj Close, Length: 7818, dtype: float64

In [ ]:
# calculates the daily percentage change for each of the selected columns, 'Adj Close' and 'Close'.
df=SPY[['Adj Close','Close']].pct_change()
df.head(10)

,Adj Close,Close
Date,,
1993-01-29,NaN,NaN
1993-02-01,0.007112,0.007112
1993-02-02,0.002119,0.002119
1993-02-03,0.010571,0.010571
1993-02-04,0.004184,0.004184
1993-02-05,-0.000694,-0.000694
1993-02-08,0.000000,0.000000
1993-02-09,-0.006950,-0.006949
1993-02-10,0.001400,0.001400


In [ ]:
# Looks like the difference is due to splits and dividends. Divs are quarterly.
df.loc[(df['Adj Close']-df['Close'])>1E-6]

,Adj Close,Close
Date,,
1993-03-19,-0.001513,-0.006207
1993-06-08,-0.009002,-0.009003
1993-06-18,-0.008236,-0.015214
1993-08-31,0.002693,0.002692
1993-09-17,-0.001939,-0.008119
...,...,...
2022-12-16,-0.011806,-0.016323
2023-03-17,-0.011693,-0.015450
2023-06-16,-0.003406,-0.007094


## Let's plot daily returns


In [ ]:
SPY['Adj Close'].pct_change().plot()

Observations?

- There are periods where the market is more volatile. calm periods and volatile periods.
- 1997 financial crisis
- 1998 what happened? There was a long term capital management blow-up. There was a hedge fund called Long Term Capital Management. They owned a lot of bonds that were illiquid. So if they go bankrupt, it's going to cause severe destruction in the economy, they got bailed-out.
- 2000 dot-com bubble where tech companies were worth too much, caused a lot of volatility. the dot com bubble didn't burst until Oct 2002. It is interesting to know that (we can also see this in the graph above) the volatility peaked in July, but the market then went to bottom after July. There was a period of peak panic when everything was dropping lower and lower and lower but nobody was actually selling, adn then a year from 2002 was the best time to buy.
- March 2003, pandemic
- before 2008, there was a "smooth" period when people are buying multiple houses, by just relying on income they could get the loans anyways.
- 2008 financial crisis, volatility peaked in Oct 2008. Peak panic,following by a period where people were slow to sell or selling at small volumes.
  - Volatility is the double-edge sword. we can see that the best days of the market is also around the time of the worst days of the market.
  - Best advice is probably to cut your position to somewhere you can still sleep at night but not be completely out, because you don't want to miss the best day of the market of the last 30 years just by avoiding the worst days. plus, you might already got hit by the worst days.
  - May 2010, there was a [flash crash](https://www.theguardian.com/business/2015/apr/22/2010-flash-crash-new-york-stock-exchange-unfolded) that spooked the market for 36 minutes.
- 2011 European financial crisis, Brexit
- 2020 COVID Pandemic

Let's calculate the volatility of SPY daily returns. (use Standard deviation)

* why use standard deviation? the easiest function that makes a negative number positive is to square it. we can use absolute value but we cannot take the derivative of that. We want to make it computationally easy. so we square all deviations from the mean.

In [ ]:
SPY['Adj Close'].pct_change().std()

0.011808915765460493

the unit is %. 0.98%. What does that mean for you in terms of your 401k?

Bell curve, Gauss, normal distribution. When your sample size is big enough, it will become closer and closer to a normal distribution.

if you multiple the std by 2, which is 1.97%, you expect that 95% of the time you'd either be up 1.97% or down 1.97&. if you have invested 100 dollars, and cannot accept losing 1.97 dollars, that means you've invested too much.

# Let's calculate the Rolling 1 Month (21 days) volatility of the S&P 500 ETF, SPY


To calculate the 21 day volatility, you just slice a 21 day window within the period, calculate the s.t.d, move 1 day forward, calculate again, and so on.

In [ ]:
SPY['Adj Close'].pct_change().rolling(21).std().head(30)

Date
1993-01-29         NaN
1993-02-01         NaN
1993-02-02         NaN
1993-02-03         NaN
1993-02-04         NaN
1993-02-05         NaN
1993-02-08         NaN
1993-02-09         NaN
1993-02-10         NaN
1993-02-11         NaN
1993-02-12         NaN
1993-02-16         NaN
1993-02-17         NaN
1993-02-18         NaN
1993-02-19         NaN
1993-02-22         NaN
1993-02-23         NaN
1993-02-24         NaN
1993-02-25         NaN
1993-02-26         NaN
1993-03-01         NaN
1993-03-02    0.008233
1993-03-03    0.008150
1993-03-04    0.008266
1993-03-05    0.007969
1993-03-08    0.009321
1993-03-09    0.009362
1993-03-10    0.009365
1993-03-11    0.009235
1993-03-12    0.009556
Name: Adj Close, dtype: float64

Why is there NaN here?

- Because we are rolling 21, it cannot compute the first 20 days' s.t.d.



In [ ]:
SPY['Adj Close'].pct_change().rolling(21).std().plot()

## We assumed that Yahoo Finance did not miss any days. But we don't know that.

* When days are missing, we would be calculating more than 21 business days, which would be a problem if using this data to price financial contracts that have a stated number of days.
* Let's ensure the index has all business days.
* If prices are missing, forward fill (ffill)


In [ ]:
# We first need to define what we mean by "business days"
# in this case, including holidays, weekdays not weekends.
pd.bdate_range(SPY.index[0],SPY.index[-1]) #Our list of business days

DatetimeIndex(['1993-01-29', '1993-02-01', '1993-02-02', '1993-02-03',
               '1993-02-04', '1993-02-05', '1993-02-08', '1993-02-09',
               '1993-02-10', '1993-02-11',
               ...
               '2024-02-02', '2024-02-05', '2024-02-06', '2024-02-07',
               '2024-02-08', '2024-02-09', '2024-02-12', '2024-02-13',
               '2024-02-14', '2024-02-15'],
              dtype='datetime64[ns]', length=8100, freq='B')

In [ ]:
SPY.index #What Yahoo Finance Returns. (Which excludes holidays)

DatetimeIndex(['1993-01-29', '1993-02-01', '1993-02-02', '1993-02-03',
               '1993-02-04', '1993-02-05', '1993-02-08', '1993-02-09',
               '1993-02-10', '1993-02-11',
               ...
               '2024-02-02', '2024-02-05', '2024-02-06', '2024-02-07',
               '2024-02-08', '2024-02-09', '2024-02-12', '2024-02-13',
               '2024-02-14', '2024-02-15'],
              dtype='datetime64[ns]', name='Date', length=7818, freq=None)

In [ ]:
#Let's just assume we want our list for whatever reason.
#Not all holidays are announced, such as a president's funeral, and so it might not be stipulated as a holiday in a financial contract.
#After 9/11/2001, the exchange was closed for a few days.

SPY=SPY.reindex(pd.bdate_range(SPY.index[0],SPY.index[-1]))
SPY

,Open,High,Low,Close,Adj Close,Volume
1993-01-29,43.968750,43.968750,43.750000,43.937500,24.840683,1003200.0
1993-02-01,43.968750,44.250000,43.968750,44.250000,25.017357,480500.0
1993-02-02,44.218750,44.375000,44.125000,44.343750,25.070356,201300.0
1993-02-03,44.406250,44.843750,44.375000,44.812500,25.335382,529400.0
1993-02-04,44.968750,45.093750,44.468750,45.000000,25.441381,531500.0
...,...,...,...,...,...,...
2024-02-09,498.839996,501.649994,498.489990,501.200012,501.200012,63979400.0
2024-02-12,501.170013,503.500000,500.239990,500.980011,500.980011,56502300.0
2024-02-13,494.529999,497.089996,490.720001,494.079987,494.079987,113099200.0
2024-02-14,496.790009,499.070007,494.399994,498.570007,498.570007,68387800.0


In [ ]:
#What are nan?
SPY.loc[SPY['Adj Close'].isna()]

,Open,High,Low,Close,Adj Close,Volume
1993-02-15,NaN,NaN,NaN,NaN,NaN,NaN
1993-04-09,NaN,NaN,NaN,NaN,NaN,NaN
1993-05-31,NaN,NaN,NaN,NaN,NaN,NaN
1993-07-05,NaN,NaN,NaN,NaN,NaN,NaN
1993-09-06,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2023-09-04,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-23,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-25,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01,NaN,NaN,NaN,NaN,NaN,NaN


## Stock markets to close Tuesday for Ford funeral
https://www.nbcnews.com/id/wbna16384589

* '2007-01-01' <- Expected New Year Holiday
* '2007-01-02' <- Unexpected New Year Holiday

In [ ]:
idx=SPY.loc[SPY['Adj Close'].isna()].index
idx[idx>'2006-12-20']

DatetimeIndex(['2006-12-25', '2007-01-01', '2007-01-02', '2007-01-15',
               '2007-02-19', '2007-04-06', '2007-05-28', '2007-07-04',
               '2007-09-03', '2007-11-22',
               ...
               '2023-02-20', '2023-04-07', '2023-05-29', '2023-06-19',
               '2023-07-04', '2023-09-04', '2023-11-23', '2023-12-25',
               '2024-01-01', '2024-01-15'],
              dtype='datetime64[ns]', length=160, freq=None)

## Forward Fill (Impute missing prices with previous day)


A forward fill would copy over the previous value to the next NaN.

In [ ]:
SPY['Adj Close']=SPY['Adj Close'].ffill()

In [ ]:
SPY.loc[SPY['Adj Close'].isna()]

,Open,High,Low,Close,Adj Close,Volume


In [ ]:
SPY.loc[SPY['Close'].isna()]

,Open,High,Low,Close,Adj Close,Volume
1993-02-15,NaN,NaN,NaN,NaN,25.211691,NaN
1993-04-09,NaN,NaN,NaN,NaN,25.153250,NaN
1993-05-31,NaN,NaN,NaN,NaN,25.685797,NaN
1993-07-05,NaN,NaN,NaN,NaN,25.563929,NaN
1993-09-06,NaN,NaN,NaN,NaN,26.529274,NaN
...,...,...,...,...,...,...
2023-09-04,NaN,NaN,NaN,NaN,447.788574,NaN
2023-11-23,NaN,NaN,NaN,NaN,453.182617,NaN
2023-12-25,NaN,NaN,NaN,NaN,473.649994,NaN
2024-01-01,NaN,NaN,NaN,NaN,475.309998,NaN


# Let's calculate the Rolling 1 Month (21 days) volatility of the S&P 500 ETF, SPY


In [ ]:
SPY['Adj Close'].pct_change().rolling(21).std().plot()

## Seasonality analysis - Are Mondays more volatile than other days?



How do we determine if volatility is higher on certain days?
- group all the returns on Mondays, and then take the standard deviation
- group all the returns on Tuesdays, take the standard deviation
- and so on...

In [ ]:
SPY['Adj Close'].pct_change().groupby(SPY.index.map(lambda dt:dt.weekday)).std()

0    0.012436
1    0.011810
2    0.011199
3    0.011624
4    0.010887
Name: Adj Close, dtype: float64

Are Mondays more volatile than Fridays?

In [ ]:
# we use SPY['Adj Close'].pct_change() to calculate the returns first before Groupby
SPY['Adj Close'].pct_change().groupby(SPY.index.map(lambda dt:dt.weekday)).std().plot.bar()

0 - Monday

1 - Tuesday

2 - Wednesday

3 - Thursday

4 - Friday

So Monday isn't necessarily a LOT more volatile than other weekdays.

## What if we reindex by calendar days, not week days?


In [ ]:
SPY=SPY.reindex(pd.date_range(SPY.index[0],SPY.index[-1]))
SPY

,Open,High,Low,Close,Adj Close,Volume
1993-01-29,43.968750,43.968750,43.750000,43.937500,24.840683,1003200.0
1993-01-30,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-01,43.968750,44.250000,43.968750,44.250000,25.017357,480500.0
1993-02-02,44.218750,44.375000,44.125000,44.343750,25.070356,201300.0
...,...,...,...,...,...,...
2024-02-11,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-12,501.170013,503.500000,500.239990,500.980011,500.980011,56502300.0
2024-02-13,494.529999,497.089996,490.720001,494.079987,494.079987,113099200.0
2024-02-14,496.790009,499.070007,494.399994,498.570007,498.570007,68387800.0


In [26]:
SPY['Adj Close']=SPY['Adj Close'].ffill()

* Now plot 1M = 30 cdays rolling volatility

In [27]:
SPY['Adj Close'].pct_change().rolling(30).std().plot()

In [ ]:
SPY['Adj Close'].pct_change().groupby(SPY.index.map(lambda dt:dt.weekday)).std()

0    0.012500
1    0.011858
2    0.011230
3    0.011663
4    0.010930
5    0.000000
6    0.000000
Name: Adj Close, dtype: float64